## Introduction
This IPython notebook explains a basic workflow two tables using py_entitymatching. Our goal is to come up with a workflow to match books from Amazon and Walmart sites. Specifically, we want to achieve precision of at least 90% and recall as high as possible.  The datasets contain information about books.

In [2]:
import py_entitymatching as em
import pandas as pd
import os, sys

### Reading the input tables

In [3]:
A = em.read_csv_metadata('amazon_books_utf8.csv')
B = em.read_csv_metadata('walmart_books_utf8.csv')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
A['id'] = range(0, len(A))
em.set_key(A, 'id')
B['id'] = range(0, len(B))
em.set_key(B, 'id')

True

In [5]:
sep1 = '–'
sep2 = ':'
sep3 = ')'
sep4 = '('
A['Name'] = A['Name'].apply(lambda x: x.split(sep1, 1)[0])
A['Name'] = A['Name'].apply(lambda x: x.split(sep2, 1)[0])
A['Name'] = A['Name'].apply(lambda x: x.split(sep3, 1)[0])
A['Name'] = A['Name'].apply(lambda x: x.replace(sep4, ''))
A['Publisher'] = A['Publisher'].apply(lambda x: x.split(sep4, 1)[0])
B['Name'] = B['Name'].apply(lambda x: x.split(sep2, 1)[0])

## Blocking to generate candidate pairs
We are applying the following blockers to get rid of obvious non-matches
1. Overlap blocker : We use overlap blocker on Author and then Name 
2. Rule-based blocker: We use rule-based blocker on Pages and Sale Price

In [11]:
#Overlap Rules
ob = em.OverlapBlocker()
ob.stop_words.append('of')
C2 = ob.block_tables(A, B, 'Author', 'Author',word_level=True, overlap_size=2, l_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'], r_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'] )
#C1 = ob.block_tables(A, B, 'Author', 'Author',word_level=True, overlap_size=2, l_output_attrs=['Name', 'Author', 'ISBN10'], r_output_attrs=['Name','Author', 'ISBN10'] )
C3 = ob.block_candset(C2, 'Name', 'Name',overlap_size=2,word_level=True, rem_stop_words=True)
C4 = em.combine_blocker_outputs_via_union([C1,C3])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [18]:
#Rule based Rules
block_f = em.get_features_for_blocking(A, B, validate_inferred_attr_types=False)
rb = em.RuleBasedBlocker()
rb.add_rule(['Pages_Pages_anm(ltuple, rtuple) < 0.8'], block_f)
C5 = rb.block_candset(C4, show_progress=True)
rb.add_rule(['Sale_Price_Sale_Price_anm(ltuple, rtuple) < 0.68'], block_f)
C5 = rb.block_candset(C5, show_progress=True)

Column Weight does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column Weight
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [13]:
C5.to_csv('blocked_pairs.csv', index = False, sep = ',')

### Debug blocker output
We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now

In [20]:
 dbg = em.debug_blocker(C5, A, B)

In [21]:
dbg.head(3)

,_id,ltable_id,rtable_id,ltable_Name,ltable_Category,ltable_Author,ltable_ISBN10,ltable_Publisher,ltable_Language,ltable_Dimensions,ltable_Weight,rtable_Name,rtable_Category,rtable_Author,rtable_ISBN10,rtable_Publisher,rtable_Language,rtable_Dimensions,rtable_Weight
0,0,979,2346,Babylon's Ashes The Expanse,Books > Science Fiction & Fantasy > Science Fiction,James S. A. Corey,316217646,Orbit; Reprint edition,English,6 x 1.5 x 9.2 inches,1.3 pounds,The Expanse,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > Action & ...,James S. A. Corey,6311296,Orbit,English,9.25 x 6.00 x 4.75 Inches (US),NaN
1,1,1952,2346,Nemesis Games The Expanse,Books > Science Fiction & Fantasy > Science Fiction,James S. A. Corey,316334715,Orbit; Reprint edition,English,6 x 1.5 x 9.2 inches,1.4 pounds,The Expanse,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > Action & ...,James S. A. Corey,6311296,Orbit,English,9.25 x 6.00 x 4.75 Inches (US),NaN
2,2,342,2346,The Expanse Boxed Set,Books > Science Fiction & Fantasy > Science Fiction,James S. A. Corey,316311294,Orbit; Box edition,English,6 x 4.8 x 9.2 inches,4.8 pounds,The Expanse,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > Action & ...,James S. A. Corey,6311296,Orbit,English,9.25 x 6.00 x 4.75 Inches (US),NaN


## Matching tuple pairs in the candidate set
In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes. This typically involves the following steps:

1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

### Sampling and labeling the candidate set
First, we randomly sample 300 tuple pairs for labeling purposes.<br>
Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [79]:
S = em.sample_table(C5, 300)
G = em.label_table(S, label_column_name='gold_labels')
G.to_csv('labeled_data.csv', index = False, sep = ',')

Column name (gold_labels) is not present in dataframe


Since, we have already labelled a sample of 300 from the blocked pairs, we will just load that into this notebook

In [26]:
path_G = 'labeled_data.csv'
G = em.read_csv_metadata(path_G,key='_id',low_memory=False,ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.


### Splitting the labeled data into development and evaluation set
In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [29]:
train_test = em.split_train_test(G, train_proportion=0.5,random_state=0)
I = train_test['train']
J = train_test['test']

I.to_csv('train.csv')
J.to_csv('test.csv')

### Selecting the best learning-based matcher
Selecting the best learning-based matcher typically involves the following steps:

1. Creating features
2. Converting the development set into feature vectors
3. Creating a set of learning-based matchers
4. Selecting the best learning-based matcher using k-fold cross validation

#### Creating features
Next, we need to create a set of features for the development set. Using automatic feature generation in py_entitymatching, set of features F is generated based on the attributes in the input tables.

We removed features that take ‘id’, ‘Rating’ and ‘Dimensions’ as parameters from F as it does not contribute effectively to decide the matching between the tuple pairs in A and B

In [31]:
F = em.get_features_for_matching(A, B)
print (F.feature_name)
F = F.drop(F.index[[44,45,46,47,48,49,50,51,52,53,54,55,56]])


Column Weight does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column Weight


The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,Name,Name,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
1,Sale Price,Sale Price,numeric,numeric,Exact Match; Absolute Norm
2,Category,Category,medium string (5 words to 10 words),short string (1 word),Not Applicable: Types do not match
3,Author,Author,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,ISBN10,ISBN10,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
5,Pages,Pages,numeric,numeric,Exact Match; Absolute Norm
6,Publisher,Publisher,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Language,Language,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
8,Dimensions,Dimensions,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
9,Weight,Weight,short string (1 word to 5 words),un-determined type,Not Applicable: Types do not match


Do you want to proceed? (y/n):y
0                     Name_Name_jac_qgm_3_qgm_3
1                 Name_Name_cos_dlm_dc0_dlm_dc0
2                 Name_Name_jac_dlm_dc0_dlm_dc0
3                                 Name_Name_mel
4                            Name_Name_lev_dist
5                             Name_Name_lev_sim
6                                 Name_Name_nmw
7                                  Name_Name_sw
8                     Sale_Price_Sale_Price_exm
9                     Sale_Price_Sale_Price_anm
10               Sale_Price_Sale_Price_lev_dist
11                Sale_Price_Sale_Price_lev_sim
12                Author_Author_jac_qgm_3_qgm_3
13            Author_Author_cos_dlm_dc0_dlm_dc0
14            Author_Author_jac_dlm_dc0_dlm_dc0
15                            Author_Author_mel
16                       Author_Author_lev_dist
17                        Author_Author_lev_sim
18                            Author_Author_nmw
19                             Author_Author_sw
20      

#### Converting the development set to feature vectors

In [33]:
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='gold_labels',
                            show_progress=False) 

# Display first few rows
H.head()

,_id,ltable_id,rtable_id,Name_Name_jac_qgm_3_qgm_3,Name_Name_cos_dlm_dc0_dlm_dc0,Name_Name_jac_dlm_dc0_dlm_dc0,Name_Name_mel,Name_Name_lev_dist,Name_Name_lev_sim,Name_Name_nmw,...,Publisher_Publisher_lev_sim,Publisher_Publisher_nmw,Publisher_Publisher_sw,Language_Language_lev_dist,Language_Language_lev_sim,Language_Language_jar,Language_Language_jwn,Language_Language_exm,Language_Language_jac_qgm_3_qgm_3,gold_labels
210,1064,1840,94,0.027778,0.000000,0.000000,0.563636,18.0,0.181818,-7.0,...,0.233333,-16.0,6.0,0.0,1.0,1.0,1.0,1.0,1.0,1
19,111,94,199,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,17.0,...,0.187500,4.0,14.0,0.0,1.0,1.0,1.0,1.0,1.0,1
254,1268,2448,1016,0.354167,0.577350,0.333333,0.877273,27.0,0.386364,-10.0,...,0.258065,-15.0,7.0,0.0,1.0,1.0,1.0,1.0,1.0,1
241,1208,2190,1977,0.161290,0.377964,0.222222,0.614268,33.0,0.250000,-9.0,...,0.464286,4.0,10.0,0.0,1.0,1.0,1.0,1.0,1.0,0
266,1336,2788,905,0.441176,0.707107,0.500000,0.900000,15.0,0.500000,0.0,...,0.333333,-5.0,7.0,0.0,1.0,1.0,1.0,1.0,1.0,1


We imputed missing values for feature vectors with 0.

In [34]:
any(pd.notnull(H))

True

In [35]:
H.fillna(value=0, inplace=True)

#### Creating a set of learning-based matchers

In [36]:
dt = em.DTMatcher(name='DecisionTree', random_state=0,max_depth=5)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

#### Selecting the best matcher using cross-validation
Now, we select the best matcher using 5-fold cross-validation. We used 'precision' and 'recall' metric and found Linear Regression(X) as the best matcher.

In [37]:
result_f1 = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
        k=5,
        target_attr='gold_labels', metric_to_select_matcher='f1', random_state=0)
result_f1['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.878323,0.861616,0.868023
1,RF,0.859974,0.873593,0.863892
2,SVM,0.748186,0.990909,0.851043
3,LinReg,0.905514,0.928571,0.915723
4,LogReg,0.906032,0.919481,0.911648
5,NaiveBayes,0.901930,0.855844,0.877045


#### Debugging Matcher

In [154]:
# Running The matcher

### Evaluating the matching output
Evaluating the matching outputs for the evaluation set typically involves the following four steps:

1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

#### Converting the evaluation set to feature vectors
As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [43]:
L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='gold_labels', show_progress=False)

In [44]:
any(pd.notnull(L))
L.fillna(value=0, inplace=True)

In [45]:
L

,_id,ltable_id,rtable_id,Name_Name_jac_qgm_3_qgm_3,Name_Name_cos_dlm_dc0_dlm_dc0,Name_Name_jac_dlm_dc0_dlm_dc0,Name_Name_mel,Name_Name_lev_dist,Name_Name_lev_sim,Name_Name_nmw,...,Publisher_Publisher_lev_sim,Publisher_Publisher_nmw,Publisher_Publisher_sw,Language_Language_lev_dist,Language_Language_lev_sim,Language_Language_jar,Language_Language_jwn,Language_Language_exm,Language_Language_jac_qgm_3_qgm_3,gold_labels
208,1058,1817,329,0.448276,0.707107,0.500000,0.904000,12.0,0.520000,1.0,...,0.172414,-19.0,5.0,0.0,1.0,1.0,1.0,1.0,1.0,1
188,953,1596,2194,0.551020,0.745356,0.555556,0.920000,18.0,0.600000,9.0,...,0.142857,-20.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,0
12,71,59,1143,0.800000,1.000000,1.000000,0.990476,1.0,0.952381,19.0,...,0.459459,10.0,15.0,0.0,1.0,1.0,1.0,1.0,1.0,1
221,1108,1887,1385,0.511628,0.816497,0.666667,0.911628,19.0,0.558140,5.0,...,0.187500,-14.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,1
239,1199,2181,516,0.662162,0.804400,0.647059,0.885965,65.0,0.429825,-16.0,...,0.400000,-2.0,6.0,0.0,1.0,1.0,1.0,1.0,1.0,0
136,713,1105,678,0.418182,0.566947,0.375000,0.894349,24.0,0.510204,2.0,...,0.176471,-16.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1
230,1156,2070,830,0.230769,0.534522,0.285714,0.851429,26.0,0.257143,-17.0,...,0.266667,5.0,5.0,0.0,1.0,1.0,1.0,1.0,1.0,1
206,1049,1798,634,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,24.0,...,0.228571,-14.0,6.0,0.0,1.0,1.0,1.0,1.0,1.0,1
52,275,314,2031,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,8.0,...,0.120000,-6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1
108,560,808,1817,0.531250,0.866025,0.750000,0.910345,13.0,0.551724,3.0,...,0.217391,-13.0,5.0,0.0,1.0,1.0,1.0,1.0,1.0,1


#### Training the selected matcher
Now, we train the matcher using all of the feature vectors from the development set. We use Linear Regression as the selected matcher.

In [41]:
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], 
       target_attr='gold_labels')

#### Predicting the matches
Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [46]:
# Predict on L 
predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], 
                         append=True,target_attr='predicted_labels')

#### Evaluating the predictions
Finally, we evaluate the accuracy of predicted outputs

In [47]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_labels', 'predicted_labels')
em.print_eval_summary(eval_result)

Precision : 90.11% (82/91)
Recall : 88.17% (82/93)
F1 : 89.13%
False positives : 9 (out of 91 positive predictions)
False negatives : 11 (out of 59 negative predictions)
